In [ ]:
# Alex Fernandez de Alarcon & Miquel Ferreiro Rojas

import os
import subprocess
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

### 1) Resize Video

In [ ]:
#C:/Video-Coding/S2/BBB_original.mp4

def resize_video():
    # Request the path to the video file
    video_path = input("Paste the (absolute) path of the video you want to resize: ")
    
    if not os.path.isfile(video_path):
        print("The specified video file does not exist. Please check the path and try again.")
        return

    # Extract the directory and video filename
    directory, video_name = os.path.split(video_path)

    # Change working directory to the video location
    os.chdir(directory)

    # Let the user select a target resolution
    print("Select the target resolution:")
    print("1. 480p (Standard Definition)")
    print("2. 720p (HD)")
    print("3. 1080p (Full HD)")

    choice = input("Enter the number corresponding to the resolution (1, 2, or 3) for (480p, 720p, 1080p): ")

    # Define the scale filter based on the choice
    resolution_map = {
        "1": "scale=-1:480",  # Keep aspect ratio, height = 480
        "2": "scale=-1:720",  # Keep aspect ratio, height = 720
        "3": "scale=-1:1080", # Keep aspect ratio, height = 1080
    }

    scale_filter = resolution_map.get(choice)
    if not scale_filter:
        print("Invalid choice. Please select 1, 2, or 3.")
        return

    # Construct the output filename
    output_name = f"resized_{os.path.splitext(video_name)[0]}_{choice}p.mp4"

    # Construct the FFmpeg command
    command = [
        "ffmpeg", "-y", "-i", video_name, "-vf", scale_filter, 
        "-preset", "fast", "-c:a", "copy", output_name
    ]

    # Execute the FFmpeg command
    print("Resizing the video...")
    subprocess.run(command)

    print(f"Resized video saved as: {output_name} in {directory}")

# Run the video resizing script
resize_video()


Select the target resolution:
1. 480p (Standard Definition)
2. 720p (HD)
3. 1080p (Full HD)
Resizing the video...
Resized video saved as: resized_BBB_original_2p.mp4 in C:/Video-Coding/S2


### 2) Modify Chroma Subsampling

In [6]:
#C:/Video-Coding/S2/BBB_original.mp4

def modify_chroma_subsampling():
    # Request the path to the video file
    video_path = input("Paste the (absolute) path of the video you want to modify chroma subsampling: ")
    
    if not os.path.isfile(video_path):
        print("The specified video file does not exist. Please check the path and try again.")
        return

    # Extract the directory and video filename
    directory, video_name = os.path.split(video_path)

    # Change working directory to the video location
    os.chdir(directory)

    # Let the user select the desired chroma subsampling format
    print("Select the chroma subsampling format:")
    print("1. 4:4:4 (Full chroma resolution)")
    print("2. 4:2:2 (Half horizontal chroma resolution)")
    print("3. 4:2:0 (Quarter chroma resolution)")

    choice = input("Enter the number corresponding to the format (1, 2, or 3): ")

    # Map choices to FFmpeg pixel formats
    chroma_map = {
        "1": "yuv444p",  # Full chroma
        "2": "yuv422p",  # Half horizontal chroma
        "3": "yuv420p",  # Quarter chroma
    }

    pix_fmt = chroma_map.get(choice)
    if not pix_fmt:
        print("Invalid choice. Please select 1, 2, or 3.")
        return

    # Construct the output filename
    output_name = f"chroma_{os.path.splitext(video_name)[0]}_{pix_fmt}.mp4"

    # Construct the FFmpeg command
    command = [
        "ffmpeg", "-y", "-i", video_name, "-pix_fmt", pix_fmt, 
        "-c:v", "libx264", "-crf", "18", "-preset", "fast", "-c:a", "copy", output_name
    ]

    # Execute the FFmpeg command
    print("Modifying chroma subsampling...")
    subprocess.run(command)

    print(f"Video with new chroma subsampling saved as: {output_name} in {directory}")

# Run the chroma subsampling script
modify_chroma_subsampling()


Select the chroma subsampling format:
1. 4:4:4 (Full chroma resolution)
2. 4:2:2 (Half horizontal chroma resolution)
3. 4:2:0 (Quarter chroma resolution)
Modifying chroma subsampling...
Video with new chroma subsampling saved as: chroma_BBB_original_yuv422p.mp4 in C:/Video-Coding/S2


### 3) Get Important Features 

In [9]:
#C:/Video-Coding/S2/BBB_original.mp4

def get_video_info():
    # Request the path to the video file
    video_path = input("Paste the (absolute) path of the video you want to analyze: ")
    
    if not os.path.isfile(video_path):
        print("The specified video file does not exist. Please check the path and try again.")
        return

    # FFprobe command to retrieve video stream metadata in JSON format
    command = [
        "ffprobe", "-v", "error", "-select_streams", "v:0",
        "-show_entries", "stream=width,height,codec_name,avg_frame_rate",
        "-show_entries", "format=duration,bit_rate",
        "-of", "json", video_path
    ]

    try:
        # Execute the FFprobe command and capture the output
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        metadata = json.loads(result.stdout)
        
        # Ensure the video stream exists
        if not metadata.get("streams"):
            print("No video stream found in the file.")
            return
        
        print("\nVideo Information:")
        # 1. Resolution
        width = metadata["streams"][0].get("width", "N/A")
        height = metadata["streams"][0].get("height", "N/A")
        print(f"Resolution: {width}x{height}")

        # 2. Duration
        duration = float(metadata["format"].get("duration", 0))
        print(f"Duration: {duration:.2f} seconds")

        # 3. Codec
        codec = metadata["streams"][0].get("codec_name", "N/A")
        print(f"Codec: {codec}")

        # 4. Frame rate
        avg_frame_rate = metadata["streams"][0].get("avg_frame_rate", "N/A")
        if avg_frame_rate != "N/A":
            num, den = map(int, avg_frame_rate.split("/"))
            frame_rate = num / den if den != 0 else 0
            print(f"Frame Rate: {frame_rate:.2f} fps")
        else:
            print("Frame Rate: N/A")

        # 5. Bit rate
        bit_rate = int(metadata["format"].get("bit_rate", 0))
        print(f"Bit Rate: {bit_rate / 1000:.2f} kbps")

    except json.JSONDecodeError:
        print("Error parsing metadata. Make sure FFmpeg is installed and the video path is correct.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Run the script
get_video_info()



Video Information:
Resolution: 640x360
Duration: 37.57 seconds
Codec: h264
Frame Rate: 30.03 fps
Bit Rate: 1188.08 kbps


### 4) New BBB Video

In [10]:
#C:/Video-Coding/S2/BBB_original.mp4

def process_bbb_video():
    # Request the path to the BBB video file
    bbb_path = input("Paste the (absolute) path of the BBB video file: ")

    if not os.path.isfile(bbb_path):
        print("The specified video file does not exist. Please check the path and try again.")
        return

    # Define output file names
    trimmed_video = "BBB_20s.mp4"
    audio_aac_mono = "BBB_audio_mono.aac"
    audio_mp3_stereo = "BBB_audio_stereo.mp3"
    audio_ac3 = "BBB_audio.ac3"
    final_output = "BBB_final.mp4"

    try:
        # Step 1: Cut BBB into a 20-second video
        print("Trimming the video to 20 seconds...")
        trim_command = [
            "ffmpeg", "-y", "-i", bbb_path, "-t", "20", "-c:v", "copy", "-an", trimmed_video
        ]
        subprocess.run(trim_command, check=True)
        print(f"Trimmed video saved as: {trimmed_video}")

        # Step 2: Export audio as AAC mono track
        print("Exporting AAC mono audio...")
        aac_command = [
            "ffmpeg", "-y", "-i", bbb_path, "-t", "20", "-vn", "-ac", "1", "-c:a", "aac", audio_aac_mono
        ]
        subprocess.run(aac_command, check=True)
        print(f"AAC mono audio saved as: {audio_aac_mono}")

        # Step 3: Export audio as MP3 stereo with lower bitrate
        print("Exporting MP3 stereo audio...")
        mp3_command = [
            "ffmpeg", "-y", "-i", bbb_path, "-t", "20", "-vn", "-ac", "2", "-b:a", "128k", "-c:a", "libmp3lame", audio_mp3_stereo
        ]
        subprocess.run(mp3_command, check=True)
        print(f"MP3 stereo audio saved as: {audio_mp3_stereo}")

        # Step 4: Export audio in AC3 codec
        print("Exporting AC3 audio...")
        ac3_command = [
            "ffmpeg", "-y", "-i", bbb_path, "-t", "20", "-vn", "-c:a", "ac3", audio_ac3
        ]
        subprocess.run(ac3_command, check=True)
        print(f"AC3 audio saved as: {audio_ac3}")

        # Step 5: Package everything into an MP4 container
        print("Packaging everything into a final MP4...")
        package_command = [
            "ffmpeg", "-y",
            "-i", trimmed_video,  # Video
            "-i", audio_aac_mono,  # AAC mono audio
            "-i", audio_mp3_stereo,  # MP3 stereo audio
            "-i", audio_ac3,  # AC3 audio
            "-map", "0:v:0",  # Use the video from the trimmed video
            "-map", "1:a:0",  # AAC mono
            "-map", "2:a:0",  # MP3 stereo
            "-map", "3:a:0",  # AC3
            "-c:v", "copy",  # Copy video codec
            "-c:a", "copy",  # Copy audio codecs
            final_output
        ]
        subprocess.run(package_command, check=True)
        print(f"Final MP4 saved as: {final_output}")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred while processing: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Run the script
process_bbb_video()


Trimming the video to 20 seconds...
Trimmed video saved as: BBB_20s.mp4
Exporting AAC mono audio...
AAC mono audio saved as: BBB_audio_mono.aac
Exporting MP3 stereo audio...
MP3 stereo audio saved as: BBB_audio_stereo.mp3
Exporting AC3 audio...
AC3 audio saved as: BBB_audio.ac3
Packaging everything into a final MP4...
Final MP4 saved as: BBB_final.mp4


### 5) Count Tracks

In [11]:
#C:/Video-Coding/S2/BBB_original.mp4

def count_tracks():
    # Request the path to the MP4 file
    mp4_path = input("Paste the (absolute) path of the MP4 file: ")

    if not os.path.isfile(mp4_path):
        print("The specified MP4 file does not exist. Please check the path and try again.")
        return

    # FFprobe command to fetch track information
    command = [
        "ffprobe", "-v", "error", "-show_entries", "stream=index,codec_type", "-of", "json", mp4_path
    ]

    try:
        # Execute the FFprobe command and capture the output
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        metadata = json.loads(result.stdout)

        if not metadata.get("streams"):
            print("No tracks found in the container.")
            return

        # Count video and audio tracks
        video_count = sum(1 for stream in metadata["streams"] if stream["codec_type"] == "video")
        audio_count = sum(1 for stream in metadata["streams"] if stream["codec_type"] == "audio")

        # Print the results
        print("\nMP4 Container Information:")
        print(f"Total Tracks: {len(metadata['streams'])}")
        print(f"Video Tracks: {video_count}")
        print(f"Audio Tracks: {audio_count}")

    except json.JSONDecodeError:
        print("Error parsing metadata. Make sure FFmpeg is installed and the MP4 file path is correct.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Run the script
count_tracks()



MP4 Container Information:
Total Tracks: 2
Video Tracks: 1
Audio Tracks: 1


### 6) Motion Vectors on a Video

In [ ]:
#C:/Video-Coding/S2/BBB_original.mp4

def visualize_motion_vectors():
    """Visualize motion vectors in a video using FFmpeg."""
    try:
        # Request the input video path
        input_video = input("Paste the absolute path of the input video file: ")
        
        # Validate the input file
        if not os.path.isfile(input_video):
            print("The specified video file does not exist. Please check the path and try again.")
            return

        # Define the output file name
        output_video = "motion_vectors_output.mp4"
        
        # FFmpeg command to visualize motion vectors
        command = [
            "ffmpeg", "-y", 
            "-flags2", "+export_mvs",  # Enable motion vector export
            "-i", input_video, 
            "-vf", "codecview=mv=pf+bf+bb",  # Visualize motion vectors
            "-c:v", "libx264", "-crf", "18",  # Re-encode with high quality
            output_video
        ]
        
        print("Processing video to visualize motion vectors...")
        subprocess.run(command, check=True)
        print(f"Motion vector visualization saved to: {output_video}")
    
    except subprocess.CalledProcessError as e:
        print(f"Error during motion vector visualization: {e}")


visualize_motion_vectors()


Processing video to visualize motion vectors...
Motion vector visualization saved to: motion_vectors_output.mp4


### 7) Show YUV Histogram

In [ ]:
def YUV_histogram(input_video_path, output_video_path="video_with_YUV_histogram.mp4", num_samples=300):

    #Due to the high amount of frames to analyze, only 1 out of 3 are taked into account (300 samples out of 900 in 30 sec.)
    
    
    temp_histogram_path = "YUV_histogram_at_real_time.png"

    # Load the input video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Unable to open the input video.")
        return

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the step to process only 'num_samples' frames
    step = max(1, frame_count // num_samples)

    # Configure the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_size = (frame_width * 2, frame_height)
    out = cv2.VideoWriter(output_video_path, fourcc, fps, output_size)

    processed_count = 0
    for i in range(0, frame_count, step):  
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)  # Skip to the current frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to YUV
        yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)

        # Compute the histogram for each YUV channel
        hist_Y = cv2.calcHist([yuv_frame], [0], None, [256], [0, 256])
        hist_U = cv2.calcHist([yuv_frame], [1], None, [256], [0, 256])
        hist_V = cv2.calcHist([yuv_frame], [2], None, [256], [0, 256])

        # Create a Matplotlib figure for the histogram
        plt.figure(figsize=(8, 4))
        plt.plot(hist_Y, color='y', label='Y - Luminance')
        plt.plot(hist_U, color='b', label='U - Chrominance (Blue)')
        plt.plot(hist_V, color='r', label='V - Chrominance (Red)')
        plt.legend()
        plt.title(f"YUV Histogram - Frame {processed_count}")
        plt.xlabel("Intensity Values")
        plt.ylabel("Frequency")
        plt.xlim([0, 256])

        # Save the histogram as a temporary image
        plt.savefig(temp_histogram_path)
        plt.close()

        # Read the saved histogram image
        histogram_img = cv2.imread(temp_histogram_path)

        # Resize the histogram to match the original frame's height
        histogram_img = cv2.resize(histogram_img, (frame_width, frame_height))

        # Combine the original frame and the histogram side by side
        combined_frame = np.hstack((frame, histogram_img))

        # Write the combined frame to the output video
        out.write(combined_frame)

        processed_count += 1
        print(f"Processing frame {processed_count} of {num_samples}...")

        if processed_count >= num_samples:
            break

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"YUV histogram video saved at {output_video_path}")